In [147]:
import pandas as pd
import librosa
import numpy as np




In [148]:
import tensorflow as tf
from tensorflow.keras import layers,models
from sklearn.model_selection import train_test_split

In [149]:
# Step 1: Read the CSV file containing file names and labels
csv_file_path = '/Users/dheemankumar/Desktop/FILE/audio-ai/hindi_broken_3s_audio_data.csv'
df = pd.read_csv(csv_file_path)

In [150]:
audio_data = []
labels = []





# Step 2 and 3: Load audio files and process the data with a sample rate of 22050
for index, row in df.iterrows():
    audio_file_path = '/Users/dheemankumar/Desktop/FILE/audio-ai/3sec_audio/' + row['name']  # Adjust the path as needed
    audio, sample_rate = librosa.load(audio_file_path, sr=22050)  # Load audio with a sample rate of 22050

    # Perform additional processing if needed, e.g., creating spectrograms\

    d=librosa.stft(audio)
    s_db=librosa.amplitude_to_db(np.abs(d),ref=np.max)

    s_db_with_channel = np.expand_dims(s_db, axis=-1)

    #print(s_db.shape)


    # Append the processed audio data and label to the lists
    audio_data.append(s_db_with_channel)
    labels.append(row[['english','hindi','punjabi','bangoli','noise']])

In [151]:

# Step 4: Create NumPy arrays
audio_data = np.array(audio_data)
labels = np.array(labels)


In [152]:
labels.shape

(881, 5)

In [153]:
audio_data.shape


(881, 1025, 130, 1)

In [154]:
# Create a Sequential model
model = models.Sequential()

# Add a Flatten layer to convert the input shape to a 1D tensor
model.add(layers.Flatten(input_shape=(1025,130,1)))

# Add one or more hidden layers with desired units and activation functions
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))

# Add the output layer with 7 units (since you want 7 outputs) and a suitable activation function (e.g., softmax for classification)
model.add(layers.Dense(5, activation='softmax'))

# Compile the model with an appropriate loss function, optimizer, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 133250)            0         
                                                                 
 dense_10 (Dense)            (None, 64)                8528064   
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dense_12 (Dense)            (None, 5)                 165       
                                                                 
Total params: 8530309 (32.54 MB)
Trainable params: 8530309 (32.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [155]:
X_train, X_test, y_train, y_test = train_test_split(audio_data, labels, test_size=0.2, random_state=42)

In [156]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [157]:
model.fit(X_train,y_train,epochs=2)

Epoch 1/2
22/22 [==============================] - 1s 18ms/step - loss: 574.0407 - accuracy: 0.7060
Epoch 2/2
22/22 [==============================] - 0s 12ms/step - loss: 743.5485 - accuracy: 0.7798


In [158]:
# Create a Sequential model
model2 = models.Sequential()

# Add a 2D convolutional layer with 32 filters, a 3x3 kernel, and 'relu' activation
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1025, 130, 1)))

# Add a max-pooling layer
model2.add(layers.MaxPooling2D((2, 2)))

# Add another 2D convolutional layer with 64 filters and 'relu' activation
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add another max-pooling layer
model2.add(layers.MaxPooling2D((2, 2)))

# Add a flattening layer to convert to 1D tensor
model2.add(layers.Flatten())

# Add a fully connected (dense) layer with 64 units and 'relu' activation
model2.add(layers.Dense(64, activation='relu'))

# Add the output layer with 7 units (since you want 7 outputs) and 'softmax' activation
model2.add(layers.Dense(7, activation='softmax'))

# Compile the model
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model2.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 1023, 128, 32)     320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 511, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 509, 62, 64)       18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 254, 31, 64)       0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 503936)            0         
                                                                 
 dense_13 (Dense)            (None, 64)               

In [159]:
# Assuming you have already created the model and loaded your data

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)




Epoch 1/10
22/22 [==============================] - 0s 21ms/step - loss: 407.3210 - accuracy: 0.8011 - val_loss: 2.6936 - val_accuracy: 0.9718
Epoch 2/10
22/22 [==============================] - 0s 16ms/step - loss: 4.7547 - accuracy: 0.9688 - val_loss: 4.0098 - val_accuracy: 0.9831
Epoch 3/10
22/22 [==============================] - 0s 17ms/step - loss: 1.9893 - accuracy: 0.9844 - val_loss: 71.7428 - val_accuracy: 0.5876
Epoch 4/10
22/22 [==============================] - 0s 15ms/step - loss: 505.5879 - accuracy: 0.7784 - val_loss: 932.9272 - val_accuracy: 0.8644
Epoch 5/10
22/22 [==============================] - 0s 15ms/step - loss: 370.7786 - accuracy: 0.8125 - val_loss: 116.8977 - val_accuracy: 0.8644
Epoch 6/10
22/22 [==============================] - 0s 19ms/step - loss: 92.2633 - accuracy: 0.8281 - val_loss: 250.4793 - val_accuracy: 0.8644
Epoch 7/10
22/22 [==============================] - 0s 14ms/step - loss: 90.9709 - accuracy: 0.8679 - val_loss: 24.3265 - val_accuracy: 0.83

In [160]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

6/6 [==============================] - 0s 3ms/step - loss: 58.7068 - accuracy: 0.8870
Test loss: 58.70676803588867, Test accuracy: 0.887005627155304


In [170]:
audio_file_path = '/Users/dheemankumar/Desktop/FILE/audio-ai/3sec_audio/hindi_sample_759.wav'  # Adjust the path as needed
audio, sample_rate = librosa.load(audio_file_path, sr=22050)  # Load audio with a sample rate of 22050





In [171]:

d=librosa.stft(audio)
s_db=librosa.amplitude_to_db(np.abs(d),ref=np.max)

s_db_with_channel = np.expand_dims(s_db, axis=-1)


In [172]:
new_data = tf.convert_to_tensor(audio, dtype=tf.float32)
audio_= np.array(s_db_with_channel)

In [164]:
type(audio_)

numpy.ndarray

In [165]:
new_data

<tf.Tensor: shape=(66150,), dtype=float32, numpy=
array([ 0.00295096,  0.00475461,  0.00419691, ..., -0.12367675,
       -0.11059908, -0.11454123], dtype=float32)>

In [166]:
audio_.shape

(1025, 130, 1)

In [173]:
input_data = audio_.reshape(1, 1025, 130, 1)

In [174]:
predictions = model.predict(input_data)

1/1 [==============================] - 0s 60ms/step


In [175]:
predictions

array([[1., 0., 0., 0., 0.]], dtype=float32)